# Dark Frame Calibration

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
from tifffile import imwrite


import sys
sys.path.append('../../tauSWIRcamera')  # Add the path to the directory containing file_to_import.py
from tauSWIRcamera import tauSWIRcamera

plt.rcParams["font.family"] = "Times New Roman"

In [ ]:
hostname = '129.123.5.125'
port = 4000
cam = tauSWIRcamera("Industrial", hostname, port)
cameraFolder = "CAM1"
saveTIFdata = True

## Data Acquisition

In [ ]:
def runDarkFrameAnalysis(gain, N=50):
    ### Set gain ###
    cam.setSensorGain(gain)

    # Integration time vector
    intTime_ms = np.linspace(1,30,30)
    intTime_ms = np.insert(intTime_ms, 0, 0.011)

    # FPA possible setpoint temperatures
    FPA_temp_setpoint_options = [0, 20, 40, 45] #oC


    fig, ax = plt.subplots()
    marker = {20: '-o', 40:'-s'}
    color = {20: 'blue',40:'red'}
    for fpaTemp in [20, 40]:
        # Set FPA setpoint temp
        print(f"Set FPA temp to {fpaTemp}oC")
        cam.setFPATempSetPoint(FPA_temp_setpoint_options.index(fpaTemp))
        df_mean  = np.array([])
        df_mean_estimated = np.array([])
        df_std  = np.array([])
        df_estimated_std = np.array([])
        for t_ms in intTime_ms:
            # Set Integration Time
            print(f"Set integration time to {t_ms}")
            cam.setIntTime(t_ms)
            # Collect frames
            df_stack = cam.collectFrame(N)
            # Save the stack of images as a multi-page TIFF file
            if saveTIFdata:
                imwrite(f'{cameraFolder}/raw_images_stack_{gain}_{fpaTemp}C_{cameraFolder}_expTime_{t_ms}ms.tif', df_stack)
            # Compute mean and std of the dark frame stack
            dark_frame_mean = np.mean(df_stack, axis=0)
            dark_frame_std = np.std(df_stack, axis=0)
            # Save the mean and std of dark frame 
            if saveTIFdata:
                imwrite(f'{cameraFolder}/dark_frame_{gain}_{fpaTemp}C_{cameraFolder}_expTime_{t_ms}ms.tif', dark_frame_mean)
                imwrite(f'{cameraFolder}/dark_frame_std_{gain}_{fpaTemp}C_{cameraFolder}_expTime_{t_ms}ms.tif', dark_frame_std)
            
            ### Compute Statistics
            # Subtrack the bias frame
            # bias_frame_stack = np.expand_dims(bias_frame, axis=0) # Add a new axis to make it 3D
            # df_stack_biasFree_int32 = df_stack.astype(np.int32) - bias_frame_stack.astype(np.int32)

            df_data = df_stack
            # Remove outliers
            mask_deadPixels = abs(df_data.mean(axis=0)-df_data.mean()) > 5*df_data.std()
            numDeadPixels = np.sum(mask_deadPixels)
            # Recompute mean and std ignoring the dead pixels
            # Apply the mask to the stack of arrays
            # Repeat the 2D mask along the third dimension to make it compatible with the 3D array
            mask_deadPixels_stack = np.tile(mask_deadPixels[None, :, :], (df_data.shape[0], 1, 1))
            masked_df = np.ma.masked_array(df_data, mask_deadPixels_stack)

            df_mean = np.append(df_mean, np.mean(masked_df))
            df_std = np.append(df_std, np.std(masked_df))

            df_mean_estimated = np.append(df_mean_estimated, cam.darkFrameMeanCounts())
            df_estimated_std = np.append(df_estimated_std, cam.getNoiseCount_Std())
            print(f"Num of dead pixels: {numDeadPixels}")
            print(f"mean: {round(np.mean(masked_df))}, std: {round(np.std(masked_df))}")
            print(f"FPA actual temp: {cam.getFPAtemp()}")
            print(" ")
        # Plot results
        ax.errorbar(intTime_ms, df_mean, yerr=df_std, label=f"FPA Temp. (\N{DEGREE SIGN}C): {cam.getFPAtemp()}", capsize=10, marker='o', markersize=5, fmt=marker[fpaTemp], color=color[fpaTemp])
        # Fill area between the two error bars
        plt.fill_between(intTime_ms, df_mean - df_std, df_mean + df_std, color=color[fpaTemp], alpha=0.2)


        # ax.errorbar(intTime_ms, df_mean_estimated, yerr=df_estimated_std, label=f"Estimated ({gain} gain)", capsize=10, marker='*', markersize=5)
    plt.suptitle(f"Dark frame analysis", fontsize = 14, fontweight="bold")
    plt.title(f"{cameraFolder} - Gain Mode: {gain}", fontsize = 12)
    plt.ylabel("Counts")
    plt.xlabel("Integration time (ms)")
    plt.legend()   
    plt.grid() 
    # Display the mean and standard deviation
    plt.text(0.98, 0.05, f'Error bar size: 2σ\n Num. frames per sample: {N}', horizontalalignment='right', verticalalignment='center', transform=plt.gca().transAxes, fontsize=10)
    # Save the plot as a PDF file
    plt.savefig(f'{cameraFolder}/dark_frame_analysis_{gain}_{cameraFolder}.pdf', format='pdf')
    plt.show()


In [ ]:
## LOW GAIN
runDarkFrameAnalysis("low", N=25)

In [ ]:
cam.setSensorGain("low")
cam.setFPATempSetPoint(1)
cam.setIntTime(0.011)
print(np.std(cam.collectFrame(25)))

In [ ]:
## Medium GAIN
runDarkFrameAnalysis("medium", N=25)

In [ ]:
## High GAIN
runDarkFrameAnalysis("high", N=25)